## **Import necessary libraries**

In [ ]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import random
import copy
import time as time
from torch.autograd import Variable
import numpy as np

# **Define variables for optimizer function and data processing**

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define variables for optimizer function and data processing
NUMOFCLIENTS = 10
SELECT_CLIENTS = 0.5
EPOCHS = 5
CLIENT_EPOCHS = 5
BATCH_SIZE = 10
DROP_RATE = 0

# Model config
LOSS = nn.CrossEntropyLoss()
NUMOFCLASSES = 10
lr = 0.0025
OPTIMIZER = optim.SGD  # You can customize the optimizer later if needed

# PSO Acc

ACC = 0.5
LOCAL_ACC = 0.2
GLOBAL_ACC = 0.2

PARAMS = {
    'acc': 0.5,
    'local_acc': 0.2,
    'global_acc': 0.3,
    # Add other parameters as needed
}

## Writes the data into a csv file

### Part of preprocessing

In [ ]:
def write_csv(method_name, list):
    file_name = '{name}_CIFAR10_randomDrop_{drop}%_output_C_{c}_LR_{lr}_CLI_{cli}_CLI_EPOCHS_{cli_epoch}_TOTAL_EPOCHS_{epochs}_BATCH_{batch}.csv'
    file_name = file_name.format(folder="origin_drop",drop=DROP_RATE, name=method_name, c=SELECT_CLIENTS, lr=lr, cli=NUMOFCLIENTS, cli_epoch=CLIENT_EPOCHS, epochs=EPOCHS, batch=BATCH_SIZE)
    f = open(file_name, 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)

    for l in list:
        wr.writerow(l)
    f.close()

## **Continue data preprocessing**

Prepare training and testing sets

In [ ]:
def load_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    # Extract actual data from the data loaders
    x_train, y_train = next(iter(DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)))
    x_test, y_test = next(iter(DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)))

    return (x_train, y_train), (x_test, y_test)


## **Instantiate Model**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simple CNN architecture
class FlModel(nn.Module):
    def __init__(self):
        super(FlModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(512 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 512 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize your model
model = FlModel()

# Optional: Initialize model weights if necessary
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

model.apply(weights_init)

# Save the initialized model
torch.save(model.state_dict(), 'your_model.pth')


## **Splits the training and testing sets among the available clients**

Decentralized learning

In [ ]:
def client_data_config(train_data_tuple):
    client_data = [() for _ in range(NUMOFCLIENTS)]
    num_of_each_dataset = len(train_data_tuple[0]) // NUMOFCLIENTS

    for i in range(NUMOFCLIENTS):
        split_data_index = random.sample(range(len(train_data_tuple[0])), num_of_each_dataset)
        new_x_train = torch.stack([train_data_tuple[0][k] for k in split_data_index])
        new_y_train = torch.tensor([train_data_tuple[1][k] for k in split_data_index])

        client_data[i] = (new_x_train, new_y_train)

    return client_data


## **Implementation of Particle Swarm Optimization Algorithm**

This code defines a Particle Swarm Optimization (PSO) algorithm for optimizing neural network weights in a federated learning context. Each particle corresponds to a client's model and is initialized with a unique identifier, the client model, and training data. The PSO algorithm updates the model's weights based on velocities, local best, and global best weights. The model is trained with the updated weights, and the best weights are saved. If the training loss improves, the local best model is updated. The global best model is updated if the local best score is lower than the global best score. The code allows retrieval of the best model based on particle ID. This PSO-based approach aims to optimize model performance across clients by dynamically adjusting model weights.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import random
import copy

class Particle:
    def __init__(self, particle_num, x_train, y_train):
        self.particle_id = particle_num
        self.particle_model = FlModel().to(device)
        self.local_best_model = FlModel().to(device)
        self.global_best_model = FlModel().to(device)
        self.local_best_score = float('inf')  # Initialize with positive infinity
        self.global_best_score = float('inf')  # Initialize with positive infinity

        self.x = x_train.to(device)
        self.y = y_train.to(device)

        self.parm = {'acc': 1.0, 'local_acc': 1.0, 'global_acc': 1.0}

        # Initialize velocities with random values
        self.velocities = [torch.randn_like(param) / 5 - 0.10 for param in self.particle_model.parameters()]

    def train_particle(self):
        print("particle {}/{} fitting".format(self.particle_id + 1, NUMOFCLIENTS))

        step_model = copy.deepcopy(self.particle_model)
        step_model.train()

        new_parameters = []
        for i, (new_param, param, lb_param, gb_param) in enumerate(zip(self.particle_model.parameters(), step_model.parameters(), self.local_best_model.parameters(), self.global_best_model.parameters())):
            new_v = self.parm['acc'] * self.velocities[i]
            new_v += self.parm['local_acc'] * random.random() * (lb_param - param)
            new_v += self.parm['global_acc'] * random.random() * (gb_param - param)
            self.velocities[i] = new_v
            new_parameters.append(param + self.velocities[i])

        # Extend new_parameters to match the number of parameters in the model
        while len(new_parameters) < len(list(step_model.parameters())):
            new_parameters.append(torch.zeros_like(new_parameters[0]))

        # Apply the new parameters to the model
        with torch.no_grad():
            for i, param in enumerate(step_model.parameters()):
                param.data = new_parameters[i].data

        optimizer = OPTIMIZER(step_model.parameters(), lr=lr)
        criterion = LOSS

        for epoch in range(CLIENT_EPOCHS):
            for inputs, labels in DataLoader(TensorDataset(self.x, self.y), batch_size=BATCH_SIZE, shuffle=True):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = step_model(inputs)

                # Ensure the size of outputs and labels match
                outputs = outputs[:labels.size(0), :]

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        with torch.no_grad():
            step_model.eval()
            outputs = step_model(self.x)
            loss = criterion(outputs, self.y)

        step_model.train()

        if self.global_best_score >= loss:
            self.local_best_model.load_state_dict(step_model.state_dict())
            self.local_best_score = loss.item()

        return self.particle_id, loss.item()

    def update_global_model(self, global_best_model, global_best_score):
        if self.local_best_score <= global_best_score:
            self.global_best_model.load_state_dict(global_best_model.state_dict())
            self.global_best_score = global_best_score

    def resp_best_model(self, gid):
        if self.particle_id == gid:
            return copy.deepcopy(self.particle_model)


In [ ]:
def get_best_score_by_loss(step_result):
    # step_result = [[step_model, train_socre_acc],...]
    temp_score = 100000
    temp_index = 0

    for index, result in enumerate(step_result):
        if temp_score > result[1]:
            temp_score = result[1]
            temp_index = index

    return step_result[temp_index][0], step_result[temp_index][1]


In [ ]:
def get_best_score_by_acc(step_result):
    # step_result = [[step_model, train_socre_acc],...]
    temp_score = 0
    temp_index = 0

    for index, result in enumerate(step_result):
        if temp_score < result[1]:
            temp_score = result[1]
            temp_index = index

    return step_result[temp_index][0], step_result[temp_index][1]


instantiate model

In [ ]:
def init_model():
    model = FlModel()
    optimizer = OPTIMIZER(model.parameters(), lr=lr)
    criterion = LOSS
    return model, optimizer, criterion


In [ ]:
def calculate_accuracy(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Get predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Update counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy


In [ ]:
def client_update(index, client, now_epoch, avg_state_dict, train_loader):
    print(f"client {index + 1}/{len(selected_model)} fitting")

    if now_epoch != 0:
        client.load_state_dict(avg_state_dict)

    # Define the optimizer and other necessary settings
    optimizer = OPTIMIZER(client.parameters(), lr=lr)

    for epoch in range(CLIENT_EPOCHS):
        # Your training loop here
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = client(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Print loss for each iteration
            # print(f"Client {index + 1}/{len(selected_model)} - Epoch {now_epoch + 1}/{EPOCHS} - Loss: {loss.item()}")

            accuracy = calculate_accuracy(client, train_loader)
            client.accuracy = accuracy  # Store accuracy as an attribute
            print(f"Client {index + 1}/{int(NUMOFCLIENTS * SELECT_CLIENTS)} - Epoch {epoch + 1}/{CLIENT_EPOCHS} - Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

    return client.state_dict()

## **Runs federated learning code on client datasets, displaying epochs, weighted average, loss, and accuracy.**

In [ ]:
if __name__ == "__main__":
    train_loader, test_loader = load_dataset()

    # Pass the tuple to client_data_config
    client_data = client_data_config(train_loader)

    # Extract the dataset from the tuple
    train_dataset, _ = train_loader

    pso_model = [Particle(particle_num=i, x_train=client_data[i][0], y_train=client_data[i][1]) for i in range(NUMOFCLIENTS)]

    server_evaluate_acc = []
    global_best_model = None
    global_best_score = 0.0

    for epoch in range(EPOCHS):
        server_result = []
        start = time.time()

        for particle in pso_model:
            if epoch != 0:
                particle.update_global_model(server_model, global_best_score)

            pid, train_score = particle.train_particle()
            rand = random.randint(0, 99)

            # Randomly dropped data sent to the server
            drop_communication = range(DROP_RATE)
            if rand not in drop_communication:
                server_result.append([pid, train_score])

        # Send the optimal model to each particle after the best score comparison
        gid, global_best_score = get_best_score_by_loss(server_result)
        for particle in pso_model:
            if particle.resp_best_model(gid) is not None:
                global_best_model = particle.resp_best_model(gid)

        server_model.load_state_dict(global_best_model.state_dict())

        print("server {}/{} evaluate".format(epoch + 1, EPOCHS))
        accuracy = calculate_accuracy(server_model, test_loader)
        print(f"Epoch {epoch + 1}/{EPOCHS} - Accuracy on test set: {accuracy}")
        server_evaluate_acc.append(accuracy)

    write_csv("FedPSO", server_evaluate_acc)


Files already downloaded and verified
Files already downloaded and verified
particle 1/10 fitting
